In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import kuaizi as kz
from astropy.table import Table, vstack
import astropy.units as u

from kuaizi.display import display_single
from astropy.io import fits

#from tiger_cutout_hsc import cutout_one, prepare_catalog # generate_cutout, get_psf
#import lsst.daf.persistence as dafPersist

In [5]:
kz.utils.set_env(project='HSC', name='LSBG', data_dir='/scratch/gpfs/jiaxuanl/Data/')

'/scratch/gpfs/jiaxuanl/Data/HSC/LSBG'

In [9]:
# Select 100 galaxies
lsbg_cat = Table.read('./Catalog/lsbg_NSA_z001_002_20hosts.fits') # This contains both candy+galaxy and junks

#rnd = np.random.randint(0, len(lsbg_cat), 1000)

# rnd = np.random.randint(0, len(lsbg_cat), 200)

# rnd = np.array([5639, 2575,  782,  865,  636, 1205, 3672, 6502, 2581,  531, 6369,
#        5619, 5341, 1798,  472,  742,  523, 5345, 6639, 5890, 4942, 4686,
#        2807,  161,  565, 5138, 3618,   95, 3581, 4915, 4825, 6124, 4654,
#        3804, 1685,  583, 4148, 3393, 7455, 1085, 4428, 5972, 2193,  150,
#        6056, 1109, 1008, 2546, 6907, 4958])
# cat = lsbg_cat[rnd]

In [10]:
cat = lsbg_cat#[(lsbg_cat['good_votes'] + lsbg_cat['bad_votes']) > 2]

In [11]:
len(cat)

501

### Adaptive cutout size

In [12]:
## Adaptive cutout size??  
# Normal objects, use 1.0 arcmin cutout.
# Radius > 20 arcsec, use 1.5 arcmin cutout.
# Radius > 30 arcsec, use 2.0 arcmin cutout.
size = np.ones(len(cat)) * 1.0 * u.arcmin

size[cat['flux_radius_ave_i'] > 20] = 1.5 * u.arcmin # shoud be larger
size[cat['flux_radius_ave_i'] > 30] = 2 * u.arcmin # should be larger 

cat['cutout_size'] = size.value
cat.write('./Catalog/nsa_20hosts_sample_211103.fits', overwrite=True)

In [13]:
cat

ra,dec,a_image,b_image,theta_image,ellipticity,mag_auto_g,mag_auto_r,mag_auto_i,flux_radius_ave_g,flux_radius_ave_i,mu_ave_g,mu_ave_i,acorr_ratio,g-i,g-r,A_g,A_r,A_i,tract,patch,synth_id,viz-id,is_candy,is_galaxy,is_tidal,is_outskirts,is_cirrus,is_junk,num_votes,good_votes,bad_votes,cutout_size
float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,bytes3,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,int64,float64
148.644722,0.6324236,26.481,19.073,-29.64,0.28,17.8636,17.4643,17.2559,5.299812,5.179692,22.965983582252385,22.334202242303963,1.0666763135780277,0.5990814941543471,0.4005263876679588,0.091888629160825,0.06511501682878135,0.04647012331517132,9569,"1,3",-1,29,2,0,0,0,0,0,2,2,0,1.0
148.6218765,0.7762161,16.637,13.652,22.08,0.179,21.3117,20.3191,20.2705,9.04974,11.158476000000002,27.387469794055974,27.12978041616876,1.2295926211096606,0.66058848201742,0.6362756883982565,0.1044180460811507,0.07399373447940495,0.05280652809857141,9569,"1,4",-1,71,0,0,0,0,1,1,2,0,2,1.0
177.2698169,-1.3470333,16.677,14.109,-89.87,0.154,18.246,17.9568,17.7463,3.769248,3.4680240000000007,22.621140107224814,21.898387993447997,1.1231178296395317,0.521845271199144,0.307726311307631,0.05756829675417243,0.04079460806180298,0.0291135679533164,9346,"7,1",-1,441,0,2,0,0,0,0,2,2,0,1.0
148.3725403,1.0200613,12.367,5.024,-30.94,0.594,22.3699,21.5443,21.6433,4.0249440000000005,3.2844,26.99839167413158,25.954364311900566,1.4396221128498627,0.8657919171570793,0.8350967371476666,0.09267680339872514,0.06567354054639013,0.04686872055580439,9569,"3,6",-1,571,0,0,0,1,0,1,2,0,2,1.0
148.5526387,0.811883,7.292,5.432,32.1,0.255,22.2204,21.5026,21.1455,5.051844,5.244540000000001,27.017050464307594,26.45471917337883,1.5767805868036053,0.810413385487835,0.5477956252375097,0.10537923949801004,0.07467486473552148,0.05329262498584795,9569,"2,4",-1,658,0,0,0,0,0,2,2,0,2,1.0
343.0545009,1.0592549,73.457,41.452,-31.43,0.436,17.8586,17.1306,16.6868,15.930768,16.751364000000002,25.23200920729965,24.309323325800257,1.0794807676521885,0.9561652838461318,0.6178557555588597,0.2572131022061703,0.18226885776502816,0.13007838605230077,9701,"7,6",-1,783,0,1,0,1,0,1,3,1,2,1.0
184.522397,-1.0402111,50.831,13.71,42.53,0.73,19.4873,18.9621,18.6324,5.481,6.817272,24.396628428501575,23.775435998074652,1.1609416167830167,0.7268039023484364,0.4641975444381913,0.08375067816489719,0.059348222603086746,0.042354580513333406,9351,"8,2",-1,1001,0,0,0,2,0,0,2,0,2,1.0
16.2733746,-0.9957193,14.986,11.664,-31.04,0.222,18.9533,18.5787,18.3287,3.1489920000000002,2.986872,22.883729889450308,22.13938479931992,1.2217132162921536,0.5976824913147268,0.3645364873707568,0.08601946531882873,0.060955952689587575,0.043501956633555515,9237,"4,3",-1,1083,0,2,0,0,0,0,2,2,0,1.0
148.1755984,0.7029789,4.913,4.275,-82.63,0.13,22.7233,22.142,22.4515,4.952304000000001,2.6455800000000003,27.91116732100387,27.599959820668012,1.8292946662904634,0.24540053969913048,0.35195338447846264,0.09488482800544593,0.06723821248390863,0.047985367704579056,9569,"4,4",-1,1160,0,0,0,0,0,2,2,0,2,1.0


### Generate cutout

Run `'/home/jiaxuanl/Research/Packages/kuaizi/diezi/gen_cutout/nsa_sample/lsbg_cutout_nsa.sh'`

In [ ]:
#### Download quick RGB from Johnny's website
import urllib

temp = cat
for k, idx in enumerate(temp['viz-id'].data):
    suffix = ''
    if temp[k]['is_galaxy'] >= 1:
        suffix += '_gal'
    if temp[k]['is_candy'] >= 1:
        suffix += '_cd'
    if temp[k]['bad_votes'] >= 1:
        suffix += '_jk'
        
    url = f'https://hugs-detections.nyc3.digitaloceanspaces.com/hugs-{idx}.png'
    urllib.request.urlretrieve(url, filename=f'{idx}' + suffix + '.png')